In [ ]:
import numpy as np
import torch
import pandas as pd
from ucimlrepo import fetch_ucirepo 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
Xf = iris.data.features 
yf = iris.data.targets 
  

In [ ]:
# metadata 
# variable information 
iris.variables

In [ ]:
classes=yf['class'].unique().tolist()

In [ ]:
perm = np.random.permutation(150).tolist()

In [ ]:
Xf,yf=Xf.iloc[perm],yf.iloc[perm]

In [ ]:
def encode_target(t,classes=classes):
    return classes.index(t)

In [ ]:
yf['label']=yf['class'].apply(encode_target)

In [ ]:
train_n=100

In [ ]:
X=torch.tensor(Xf.iloc[0:train_n].values, dtype=torch.float32)

In [ ]:
y=torch.tensor(yf.iloc[0:train_n]['label'].values,dtype=torch.float32)

In [ ]:
W=torch.rand(150,3)

In [ ]:
class Linear():
    def __init__(self,dims=[4,1]):
        self.W=torch.rand(dims,requires_grad=True)
        self.b=torch.rand(1,requires_grad=True)
    def __call__(self,x):
        return x@self.W+self.b


In [ ]:
model=Linear()

In [ ]:
def mse(x,y):
    return torch.mean((x-y)**2)

In [ ]:
loss=mse(model(X),y)

In [ ]:
grad_W, grad_b = torch.autograd.grad(loss, [model.W, model.b])

In [ ]:
grad_W, grad_b

In [ ]:
def train(model,X,y,lr=1e-3,epochs=1):
    for e in range(epochs):
        loss=mse(model(X),y)
        print(loss)
        grad_W, grad_b = torch.autograd.grad(loss, [model.W, model.b])
        model.W=model.W-lr*grad_W
        model.b=model.b-lr*grad_b

In [ ]:
train(model,X,y,epochs=1000)

In [ ]:
Xtest=torch.tensor(Xf.iloc[train_n:].values,dtype=torch.float32)
ytest=torch.tensor(yf.iloc[train_n:]['label'].values,dtype=torch.float32)

In [ ]:
mse(model(Xtest),ytest)

In [ ]:
def classify(x):
    if x<1: return 0
    elif x>=1 and x<2: return 1
    else: return 2

In [ ]:
yc=[classify(p) for p in model(Xtest).detach().numpy()]

In [ ]:
def class_accuracy(yc,y):
    return sum([int(p==l) for p,l in zip(yc,y)])/len(ytest)

In [ ]:
class_accuracy(yc,ytest)

In [ ]:
def one_hot(y,n=3):
    oh=torch.zeros(y.shape[0],n)
    oh[torch.arange(y.shape[0]), y.long()]=1
    return oh

In [ ]:
class LinearOH():
    def __init__(self,dims=[4,3]):
        self.W=torch.rand(dims,requires_grad=True)
        self.b=torch.rand(3,requires_grad=True)
    def __call__(self,x):
        return x@self.W+self.b

In [ ]:
model=LinearOH()

In [ ]:
def train_oh(model,X,y,lr=1e-3,epochs=1):
    for e in range(epochs):
        loss=mse(model(X),y)
        print(loss)
        grad_W, grad_b = torch.autograd.grad(loss, [model.W, model.b])
        model.W=model.W-lr*grad_W
        model.b=model.b-lr*grad_b

In [ ]:
train_oh(model,X,one_hot(y),epochs=1000)

In [ ]:
mse(model(Xtest),one_hot(ytest))

In [ ]:
def accuracy(preds,y):
    prediction=torch.argmax(preds,dim=1)
    return np.mean((prediction==y.long()).detach().numpy().astype(int))

In [ ]:
accuracy(model(X),y)

In [ ]:
def reLU(x):
    return torch.max(x, torch.tensor(0.0))

In [ ]:
class NonLinearOH():
    def __init__(self,dims=[4,3]):
        self.W=torch.rand(dims,requires_grad=True)
        self.b=torch.rand(3,requires_grad=True)
    def __call__(self,x):
        return reLU(x@self.W+self.b)

In [ ]:
model=NonLinearOH()

In [ ]:
train_oh(model,X,one_hot(y),epochs=1000)

In [ ]:
accuracy(model(X),y)

In [ ]:
accuracy(model(Xtest),ytest)

In [ ]:
def cross_entropy(preds,y):
    return -torch.sum(y*torch.log(preds))

In [ ]:
preds=model(X)

In [ ]:
preds.shape

In [ ]:
def train_new(model,X,y_oh,lossfn=cross_entropy,lr=1e-3,epochs=1):
    for e in range(epochs):
        loss=lossfn(model(X),y_oh)
        print(loss)
        grad_W, grad_b = torch.autograd.grad(loss, [model.W, model.b])
        model.W=model.W-lr*grad_W
        model.b=model.b-lr*grad_b

In [ ]:
model=NonLinearOH()

In [ ]:
train_new(model,X,one_hot(y),lr=1e-3,epochs=1000)

In [ ]:
accuracy(model(X),y)

In [ ]:
cross_entropy(model(X),y)

In [ ]:
def softmax(x):
    exp_x = torch.exp(x - torch.max(x))
    return exp_x / torch.sum(exp_x, dim=1, keepdim=True)

In [ ]:
class LinearLayer():
    def __init__(self,dims=[4,3]):
        self.W=torch.rand(dims,requires_grad=True)
        self.b=torch.rand(3,requires_grad=True)
    def __call__(self,x):
        return softmax(x@self.W+self.b)

In [ ]:
model=LinearLayer()

In [ ]:
train_new(model,X,one_hot(y),lr=1e-3,epochs=1000)

In [ ]:
accuracy(model(X),y)

In [ ]:
accuracy(model(Xtest),ytest)